#  Naver 영화 네티즌 평점  수집 - 형수, 1967

In [16]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

In [17]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=23650'
url_add = url_base + url_sub
html = urlopen(url_add)
soup = BeautifulSoup(html, 'html.parser')

## iframe의 src 주소값을 얻어야 함

In [29]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=23650&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [30]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

## 읽을 페이지 수 계산

In [31]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>2</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [32]:
count = count.find('em').string     # get_text() 대신에 string
count

'2'

In [33]:
count = int(count.replace(',',''))
count

2

In [34]:
import math
total_page = math.ceil(count/10)
total_page

1

In [70]:
url = url_base + ifr_url + '&page=1'
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
result = soup.find(class_='score_result')
lis = result.find_all('li')
lis[0].find(class_='btn_area').find('strong').get_text()


'4'

## 전체 리뷰에서 데이터 수집하기

In [115]:
point = []
review = []
date = []
nick = []
good = []
bad = []
for page in range(1, total_page+1):
    print(page, end=' ')
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    for li in lis:
        score = li.find(class_='star_score').find('em').get_text()
        score = int(score)
        point.append(score)
        count_span = len(li.find(class_='score_reple').find(class_='').find_all('span'))
        count_icon = len(li.find_all(class_='ico_viewer'))
        count_spo = len(li.find_all(class_='text_spo'))
        criteria = str(count_span) + str(count_icon) + str(count_spo)
        #print(criteria)
        if criteria == '100':
            r = li.find(class_='score_reple').find('span').string.strip(' \t\r\n')
        elif criteria == '200':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '210':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')
        elif criteria == '310':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '311':
            r = li.find(class_='score_reple').find_all('span')[2].string.strip(' \t\r\n')
        elif criteria == '201':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')

        review.append(r)
        date.append(li.find(class_='score_reple').find_all('em')[1].get_text())
        nick.append(li.find(class_='score_reple').find_all('span')[1].get_text())
        good.append(li.find(class_='_sympathyButton').find('strong').get_text())
        bad.append(li.find(class_='_notSympathyButton').find('strong').get_text())
 

1 

In [80]:
good

['4', '0']

In [81]:
bad

['0', '0']

In [106]:
point

[10, 10]

In [121]:
star = []
for i in range(len(point)):
    star.append('★' * int(point[i]/2))
star

['★★★★★', '★★★★★']

In [122]:
import pandas as pd
df = pd.DataFrame({'별점':star, '평점':point, '일시':date, '닉네임':nick, '감상평':review, '좋아요':good, '싫어요':bad})
df.tail(10)

,별점,평점,일시,닉네임,감상평,좋아요,싫어요
0,★★★★★,10,2019.09.20 22:11,쿠챙챙(siyo****),등급 어딨노 이거 뭐노 이거 영화노 어뜨케 보노,4,0
1,★★★★★,10,2020.02.23 18:30,나윤이(rlas****),우왈 남진 잘생겼다..,0,0


In [124]:
df.to_csv('형수.csv')

In [125]:
len(df)

2